# Génération de la Soumission - Détection de Fraude

## Objectifs de ce notebook
1. **Charger le meilleur modèle** identifié lors de l'exploration
2. **Préparer les données de test** avec toutes les transformations
3. **Générer les prédictions** sur le test set
4. **Créer le fichier submission.csv** au format requis

---


## 1. Import des bibliothèques


In [ ]:
# Bibliothèques standards
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Gestion du déséquilibre
try:
    from imblearn.over_sampling import SMOTE
    IMBLEARN_AVAILABLE = True
except ImportError:
    print("imbalanced-learn non disponible")
    IMBLEARN_AVAILABLE = False

# XGBoost (optionnel)
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

print("Bibliothèques importées avec succès")


**Note :** Ce bloc importe toutes les bibliothèques nécessaires pour générer la soumission finale. Il inclut les modèles de machine learning, les outils de preprocessing, et gère les cas où certaines bibliothèques ne sont pas disponibles.


## 2. Fonctions de préparation des données


In [ ]:
# Fonction pour créer les features avancées (identique à l'étape 3)
def create_advanced_features(df, is_train=True):
    """Crée des features avancées pour améliorer les performances"""
    df_adv = df.copy()
    
    # Features temporelles avancées
    df_adv['period_of_day'] = pd.cut(
        df_adv['hour_of_day'],
        bins=[0, 6, 12, 18, 24],
        labels=['Nuit', 'Matin', 'Apres-midi', 'Soir'],
        include_lowest=True
    )
    
    # Features d'interaction
    df_adv['amount_per_type'] = df_adv.groupby('type')['amount'].transform('mean')
    df_adv['amount_ratio'] = df_adv['amount'] / (df_adv['amount_per_type'] + 1)
    df_adv['amount_age_ratio'] = df_adv['amount'] / (df_adv['age'] + 1)
    
    # Features statistiques par groupe
    df_adv['avg_amount_by_day'] = df_adv.groupby('day_of_week')['amount'].transform('mean')
    df_adv['amount_vs_day_avg'] = df_adv['amount'] / (df_adv['avg_amount_by_day'] + 1)
    df_adv['avg_amount_by_hour'] = df_adv.groupby('hour_of_day')['amount'].transform('mean')
    df_adv['amount_vs_hour_avg'] = df_adv['amount'] / (df_adv['avg_amount_by_hour'] + 1)
    
    # Features de seuil
    amount_threshold = df_adv['amount'].quantile(0.95)
    df_adv['is_high_amount'] = (df_adv['amount'] > amount_threshold).astype(int)
    df_adv['is_young'] = (df_adv['age'] < 25).astype(int)
    df_adv['is_senior'] = (df_adv['age'] > 65).astype(int)
    
    # Features cycliques
    df_adv['day_sin'] = np.sin(2 * np.pi * df_adv['day_of_week'] / 7)
    df_adv['day_cos'] = np.cos(2 * np.pi * df_adv['day_of_week'] / 7)
    df_adv['hour_sin'] = np.sin(2 * np.pi * df_adv['hour_of_day'] / 24)
    df_adv['hour_cos'] = np.cos(2 * np.pi * df_adv['hour_of_day'] / 24)
    
    # Features polynomiales
    df_adv['amount_squared'] = df_adv['amount'] ** 2
    df_adv['age_squared'] = df_adv['age'] ** 2
    
    return df_adv

# Fonction pour encoder les features
def encode_features(df, is_train=True):
    """Encode les features catégorielles"""
    df_enc = df.copy()
    
    # One-Hot Encoding pour 'type'
    type_dummies = pd.get_dummies(df_enc['type'], prefix='type')
    all_types = ['type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER']
    for col in all_types:
        if col not in type_dummies.columns:
            type_dummies[col] = 0
    type_dummies = type_dummies[sorted(all_types)]
    
    # One-Hot Encoding pour 'period_of_day'
    period_dummies = pd.get_dummies(df_enc['period_of_day'], prefix='period')
    
    # Concaténation
    df_enc = pd.concat([df_enc, type_dummies, period_dummies], axis=1)
    
    return df_enc

print("Fonctions de preparation des donnees definies")


**Note :** Ce bloc définit les fonctions de préparation des données qui sont identiques à celles utilisées lors de l'entraînement. Il est crucial d'utiliser exactement les mêmes transformations pour garantir que les données de test sont traitées de la même manière que les données d'entraînement.


## 3. Chargement et préparation des données


In [ ]:
# Chargement des données
print("=" * 60)
print("CHARGEMENT DES DONNEES")
print("=" * 60)

train_df = pd.read_csv('ressources/train_prepared.csv')
test_df = pd.read_csv('ressources/test_prepared.csv')

print(f"\nTrain set: {train_df.shape}")
print(f"Test set: {test_df.shape}")

# Vérification
print(f"\nColonnes test: {list(test_df.columns)}")
if 'is_fraud' in test_df.columns:
    print("ATTENTION: is_fraud present dans le test set (ne devrait pas etre la)")
else:
    print("OK: is_fraud absent du test set")


In [ ]:
# Application des transformations
print("\n" + "=" * 60)
print("PREPARATION DES DONNEES")
print("=" * 60)

# Features avancées
train_adv = create_advanced_features(train_df, is_train=True)
test_adv = create_advanced_features(test_df, is_train=False)

# Encodage
train_enc = encode_features(train_adv, is_train=True)
test_enc = encode_features(test_adv, is_train=False)

print(f"\nTrain avec features: {train_enc.shape}")
print(f"Test avec features: {test_enc.shape}")

# Sélection des features (identique à l'étape 3)
features_to_exclude = [
    'transaction_id',
    'customer_id',
    'type',
    'age_group',
    'period_of_day',
    'is_fraud'
]

feature_columns = [col for col in train_enc.columns 
                  if col not in features_to_exclude]

# Préparation des datasets
X = train_enc[feature_columns].copy()
y = train_enc['is_fraud'].copy()
X_test = test_enc[feature_columns].copy()

print(f"\nFeatures selectionnees: {len(feature_columns)}")
print(f"X shape: {X.shape}")
print(f"X_test shape: {X_test.shape}")

# Vérification de la cohérence
if list(X.columns) == list(X_test.columns):
    print("\nLes colonnes sont identiques entre train et test")
else:
    print("\nATTENTION: Differences dans les colonnes!")
    missing_in_test = set(X.columns) - set(X_test.columns)
    missing_in_train = set(X_test.columns) - set(X.columns)
    if missing_in_test:
        print(f"Colonnes manquantes dans test: {missing_in_test}")
    if missing_in_train:
        print(f"Colonnes en trop dans test: {missing_in_train}")


**Note :** Ce bloc charge les données d'entraînement et de test, puis applique toutes les transformations nécessaires (features avancées, encodage) de manière identique à l'entraînement. Il vérifie que les colonnes sont cohérentes entre train et test pour éviter les erreurs lors de la prédiction.


## 4. Entraînement du meilleur modèle


In [ ]:
# Division train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("=" * 60)
print("ENTRAINEMENT DU MEILLEUR MODELE")
print("=" * 60)

# Application de SMOTE si disponible
if IMBLEARN_AVAILABLE:
    smote = SMOTE(random_state=42, k_neighbors=5)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    print(f"\nSMOTE applique - Train shape: {X_train_smote.shape}")
else:
    X_train_smote = X_train
    y_train_smote = y_train
    print("\nSMOTE non disponible - utilisation des donnees originales")

# Entraînement du meilleur modèle (Random Forest avec paramètres optimisés)
# Vous pouvez ajuster ces paramètres selon vos résultats de l'étape 3
print("\nEntraînement du Random Forest...")
final_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

final_model.fit(X_train_smote, y_train_smote)
print("Entraînement terminé!")

# Évaluation rapide sur la validation
from sklearn.metrics import f1_score, precision_score, recall_score
y_val_pred = final_model.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred)
val_recall = recall_score(y_val, y_val_pred)

print(f"\nPerformance sur validation:")
print(f"   - F1-Score: {val_f1:.4f}")
print(f"   - Precision: {val_precision:.4f}")
print(f"   - Recall: {val_recall:.4f}")


**Note :** Ce bloc entraîne le meilleur modèle identifié lors de l'exploration (ici Random Forest avec paramètres optimisés). Il applique SMOTE si disponible pour gérer le déséquilibre, puis évalue rapidement le modèle sur la validation pour vérifier les performances avant de générer les prédictions finales.


## 5. Génération des prédictions et création de submission.csv


In [ ]:
# Prédictions sur le test set
print("=" * 60)
print("GENERATION DES PREDICTIONS")
print("=" * 60)

# Vérification finale de la cohérence des colonnes
if list(X_train.columns) == list(X_test.columns):
    # Génération des prédictions
    test_predictions = final_model.predict(X_test)
    test_proba = final_model.predict_proba(X_test)[:, 1]
    
    print(f"\nNombre de predictions: {len(test_predictions)}")
    print(f"Nombre de fraudes predites: {test_predictions.sum()}")
    print(f"Taux de fraude predit: {test_predictions.mean()*100:.2f}%")
    print(f"Distribution des probabilites:")
    print(f"   - Min: {test_proba.min():.4f}")
    print(f"   - Max: {test_proba.max():.4f}")
    print(f"   - Moyenne: {test_proba.mean():.4f}")
    
    # Création du DataFrame de soumission
    submission = pd.DataFrame({
        'transaction_id': test_df['transaction_id'],
        'is_fraud': test_predictions
    })
    
    # Vérification du format
    print(f"\nFormat de submission:")
    print(f"   - Shape: {submission.shape}")
    print(f"   - Colonnes: {list(submission.columns)}")
    print(f"   - Types: {submission.dtypes.to_dict()}")
    
    # Vérification des valeurs
    print(f"\nValeurs dans is_fraud:")
    print(submission['is_fraud'].value_counts().sort_index())
    
    # Sauvegarde
    submission.to_csv('submission.csv', index=False)
    print(f"\nFichier submission.csv cree avec succes!")
    print(f"   - Emplacement: submission.csv")
    print(f"   - Nombre de lignes: {len(submission)}")
    
    # Aperçu
    print(f"\nApercu des 10 premieres lignes:")
    display(submission.head(10))
    
    # Aperçu des fraudes prédites
    fraud_predictions = submission[submission['is_fraud'] == 1]
    if len(fraud_predictions) > 0:
        print(f"\nApercu des fraudes predites (10 premieres):")
        display(fraud_predictions.head(10))
    
else:
    print("ERREUR: Les colonnes ne correspondent pas!")
    print(f"Train columns ({len(X_train.columns)}): {list(X_train.columns)[:5]}...")
    print(f"Test columns ({len(X_test.columns)}): {list(X_test.columns)[:5]}...")


**Note :** Ce bloc génère les prédictions finales sur le test set et crée le fichier submission.csv au format requis. Il vérifie que les colonnes sont cohérentes, affiche des statistiques sur les prédictions, et sauvegarde le fichier. Le format est exactement celui demandé : transaction_id et is_fraud (0 ou 1).


## 6. Vérification finale du fichier de soumission


In [ ]:
# Vérification finale
print("=" * 60)
print("VERIFICATION FINALE")
print("=" * 60)

# Recharger le fichier pour vérifier
submission_check = pd.read_csv('submission.csv')

print(f"\nFichier submission.csv:")
print(f"   - Shape: {submission_check.shape}")
print(f"   - Colonnes: {list(submission_check.columns)}")

# Vérifications
checks = {
    "Nombre de lignes correct": len(submission_check) == len(test_df),
    "Colonne transaction_id presente": 'transaction_id' in submission_check.columns,
    "Colonne is_fraud presente": 'is_fraud' in submission_check.columns,
    "Pas de valeurs manquantes": submission_check.isnull().sum().sum() == 0,
    "Valeurs is_fraud valides (0 ou 1)": submission_check['is_fraud'].isin([0, 1]).all(),
    "Pas de doublons transaction_id": submission_check['transaction_id'].nunique() == len(submission_check)
}

print(f"\nVerifications:")
all_ok = True
for check_name, check_result in checks.items():
    status = "OK" if check_result else "ERREUR"
    print(f"   - {check_name}: {status}")
    if not check_result:
        all_ok = False

if all_ok:
    print(f"\nToutes les verifications sont OK!")
    print(f"Le fichier submission.csv est pret pour la soumission.")
else:
    print(f"\nATTENTION: Certaines verifications ont echoue!")
    print(f"Veuillez corriger les problemes avant de soumettre.")

# Statistiques finales
print(f"\nStatistiques finales:")
print(f"   - Total de transactions: {len(submission_check)}")
print(f"   - Transactions legitimes (0): {(submission_check['is_fraud'] == 0).sum()}")
print(f"   - Transactions frauduleuses (1): {(submission_check['is_fraud'] == 1).sum()}")
print(f"   - Taux de fraude predit: {(submission_check['is_fraud'] == 1).mean()*100:.2f}%")


**Note :** Ce bloc effectue une vérification finale du fichier submission.csv pour s'assurer qu'il respecte tous les critères requis : bon nombre de lignes, colonnes correctes, pas de valeurs manquantes, valeurs valides (0 ou 1), et pas de doublons. Ces vérifications sont importantes avant de soumettre le fichier.


## 7. Résumé


In [ ]:
print("=" * 60)
print("RESUME - GENERATION DE LA SOUMISSION")
print("=" * 60)

summary = {
    "Fichier genere": {
        "Nom": "submission.csv",
        "Nombre de lignes": len(submission_check),
        "Colonnes": list(submission_check.columns)
    },
    "Predictions": {
        "Transactions legitimes (0)": int((submission_check['is_fraud'] == 0).sum()),
        "Transactions frauduleuses (1)": int((submission_check['is_fraud'] == 1).sum()),
        "Taux de fraude predit": f"{(submission_check['is_fraud'] == 1).mean()*100:.2f}%"
    },
    "Modele utilise": {
        "Type": "Random Forest",
        "F1-Score (validation)": f"{val_f1:.4f}",
        "Precision (validation)": f"{val_precision:.4f}",
        "Recall (validation)": f"{val_recall:.4f}"
    },
    "Prochaines etapes": {
        "1": "Verifier que submission.csv est bien cree",
        "2": "Soumettre le fichier selon les instructions",
        "3": "Attendre les resultats de l'evaluation"
    }
}

for key, value in summary.items():
    print(f"\n{key}:")
    if isinstance(value, dict):
        for k, v in value.items():
            print(f"   - {k}: {v}")
    else:
        print(f"   {value}")

print("\n" + "=" * 60)
print("GENERATION DE LA SOUMISSION TERMINEE")
print("=" * 60)
print("\nLe fichier submission.csv est pret pour la soumission!")


**Note :** Ce bloc final résume toutes les informations importantes sur la génération de la soumission : le fichier créé, les statistiques des prédictions, les performances du modèle utilisé, et les prochaines étapes. Le fichier submission.csv est maintenant prêt à être soumis.
